# 下一站去哪

---

## 概述

这个notebook主要对纽约出租车一年半（2019-01到2020-06）的数据进行数据清洗，分析，可视化，建模等方面的操作

数据来源：

<https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page>


数据描述：

<https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf>

数据文件夹结构

需创建文件夹并将原始数据下载到data/raw

```
├─data/     # 保存数据
│  ├─clear/ # 清洗后数据集
│  ├─diff/  # 上下车差值
│  ├─dist/  # 区域间距离
│  ├─model/ # 模型
│  └─raw/   # 原始数据
├─graph/    # 保存图片
```

导入相关包

In [ ]:
import os
import time
import math

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns

## 观察数据

In [ ]:
data = pd.read_csv("data/raw/yellow_tripdata_2020-06.csv")
data.head()

profiling分析

In [ ]:
import pandas_profiling
pfr = pandas_profiling.ProfileReport(data)
pfr.to_file('report.html')

## 数据分析

### 变量分析

对profiling得到的报告进行分析

|变量|含义|类型|待处理|
|:-|:-|:-|:-|
|VendorID|记录提供商|Categorical|缺省9.2%|
|tpep_pickup_datetime|上车时间|Datetime||
|tpep_dropoff_datetime|下车时间|Datetime||
|passenger_count|乘客数量|Categorical Number|缺省9.2%，零值2.4%|
|trip_distance|打车距离|Contiguous Number|零值3.2%，存在异常值(极大值)|
|RatecodeID|价格代码|Categorical Number|缺省9.2%|
|store_and_fwd_flag|是否保存在车辆内存中|Boolean|缺省9.2%|
|PULocationID|上车区域ID|Categorical Number||
|DOLocationID|下车区域ID|Categorical Number||
|payment_type|支付类别|Categorical Number|缺省9.2%|
|fare_amount|距离计价|Contiguous Number|存在异常值(负值，极大值)|
|extra|附加费|Contiguous Number|零值47.8%(正常)|
|mta_tax|交通税|Categorical Number|两类|
|tip_amount|小费|Contiguous Number|零值41.0%(正常)|
|tolls_amount|过路费|Contiguous Number|零值94.7%(正常)，存在异常值(极大值)|
|improvement_surcharge|改善附加费|Categorical Number|两类，存在异常值(负值)|
|total_amount|总费用|Contiguous Number|存在异常值(负值，极大值)|
|congestion_surcharge|拥挤附加费|Categorical Number|两类，存在异常值(负值)|

## 想法提出

我们可以使用的数据已在上面列出，我们通过几天的思考，决定了最后的idea

**我们想做的是合理的调度出租车，给空闲的出租车下一站应去哪里的推荐，我们主要考虑两个因素：**

1. **需求**
2. **距离**

**鉴于我们能得到的数据，保留上车时间、下车时间、上车区域、下车区域，可以统计某个时间段（定为一个小时）内下车与上车的差值（空车辆/需求量），并统计得到区域间通勤的平均旅程距离，作为我们的参考值**

## 数据清洗

鉴于我们的需求，我们保留了所需的几个数据，去重，去异常值，并添加了几个可能需要的变量

In [ ]:
def clear_data(filename, write = True):
    year_month = filename.split("_")[-1].split(".")[0]
    
    data = pd.read_csv(filename)

    print("Clearing file %s ..." % filename)
    # 保留需要使用的数据
    n_data = data[['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'trip_distance', 'PULocationID', 'DOLocationID']]

    # 去掉行进距离为0的数据
    n_data = n_data[~n_data['trip_distance'].isin([0])]

    # 计算行进时间，单位s
    n_data['duration'] = (pd.to_datetime(n_data['tpep_dropoff_datetime'] ) - pd.to_datetime(n_data['tpep_pickup_datetime'])).dt.total_seconds()

    # 去掉行进时间非正的数据
    n_data = n_data[~(n_data['duration'] <= 0)]

    # 计算行进速度，单位m/s
    n_data['speed'] = (n_data['trip_distance']/n_data['duration'])*1000

    # 去掉行进速度中过高的异常值，阈值为30m/s
    n_data = n_data[~(n_data['speed'] > 30)]

    # 去掉行进时间中过高的异常值，阈值为14400s，即4小时
    n_data = n_data[~(n_data['duration'] > 14400)]

    if write:
        n_data.to_csv("data/clear/111new_yellow_tripdata_%s.csv" % (year_month))
    print("Done!")

    return n_data

In [ ]:
for root, dirs, files in os.walk("data/raw"):
    for f in files:
        filename = os.path.join(root, f)
        start = time.time()
        clear_data(filename)
        print("Cost time: %.2fs" % (time.time() - start))
        print("--------" * 10)

## 数据获取

### 上下车差值

有1-265个区域，我们以一个小时为观测时间，该段时间内该区域下车记录减去上车记录，存在正负值（负值表示上车的车辆数更多）

In [ ]:
time_format ="%Y-%m-%d %H:%M:%S"

def cal_diff(filename, write = True):
    data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])
    year_month = filename.split("_")[-1].split(".")[0]
    
    start_time = datetime.strptime(year_month, "%Y-%m")
    end_time = start_time + relativedelta(months = 1)

    # start_time = "2020-6-1 00:00:00"
    # end_time = "2020-6-2 00:00:00"

    # start_time = datetime.strptime(start_time, time_format)
    # end_time = datetime.strptime(end_time, time_format)

    data["tpep_pickup_datetime"] = pd.to_datetime(data["tpep_pickup_datetime"])
    data["tpep_dropoff_datetime"] = pd.to_datetime(data["tpep_dropoff_datetime"])

    res_times = []
    res_datas = []

    print("Calculating file %s ..." % filename)

    while start_time < end_time:
        # print(start_time)
        # res_times.append(datetime.strftime(start_time, time_format))
        res_times.append(start_time)
        
        tmp = start_time + timedelta(hours = 1)

        # 对每个小时进行统计
        res_data = np.zeros(265, dtype=np.int)

        pickup = data.loc[(data["tpep_pickup_datetime"] > start_time) & (data["tpep_pickup_datetime"] < tmp)]["PULocationID"]
        pickup.reset_index(drop=True, inplace=True)
        dropoff = data.loc[(data["tpep_dropoff_datetime"] > start_time) & (data["tpep_dropoff_datetime"] < tmp)]["DOLocationID"]
        dropoff.reset_index(drop=True, inplace=True)
        for i in range(pickup.shape[0]):
            res_data[pickup[i] - 1] -= 1
        for i in range(dropoff.shape[0]):
            res_data[dropoff[i] - 1] += 1
        
        res_datas.append(res_data)

        start_time = tmp

    res_times = pd.DataFrame(np.array(res_times)).rename(columns = {0: 'time'})
    res_datas = pd.DataFrame(np.array(res_datas))

    res = pd.concat([res_times, res_datas], axis = 1)
    if write:
        res.to_csv("data/diff/%s.csv" % (year_month), index=False)
    print("Done!")
    return res

# cal_diff("data/clear_data/new_yellow_tripdata_2020-06.csv")

我们对数据清洗过的一年半的数据进行差值计算

In [ ]:
for root, dirs, files in os.walk("data/clear"):
    for f in files:
        filename = os.path.join(root, f)
        start = time.time()
        cal_diff(filename)
        print("Cost time: %.2fs" % (time.time() - start))
        print("--------" * 10)

对上述我们获取的数据进行可视化，动态的时间变化可以更好的帮助我们观察数据

`matplotlib画图`

下面的一些属性可以帮助我们更好的观察想要的文件

In [ ]:
filename = "data/diff/2020-06.csv" # diff文件名
frames = 100 # 展示帧数
pause = 0.1 # 暂停时间，单位s

In [ ]:
# res = cal_diff("data/clear_data/new_yellow_tripdata_2020-06.csv", write=False)
res = pd.read_csv(filename)
res.head()

In [ ]:
# 使用自己的matplotlib后端，具体可看附录
%matplotlib qt

res_datas = res.iloc[:, 1:]
res_times = res.iloc[:, 0]

# print(res_times)
# print(res_datas.index)

# 创建画布
sns.set_style("whitegrid")
fig = plt.figure(figsize=(15, 10))

plt.ion()

plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

for i in range(res.shape[0]):

    if i == frames:
        break

    plt_data = res_datas.loc[i]
    plt_data.index = plt_data.index.astype(int)

    # 正值 0-20 红色
    plt_data0 = plt_data.loc[(plt_data > 0) & (plt_data <= 20)]
    # 负值 -20-0 蓝色
    plt_data1 = plt_data.loc[(plt_data < 0) & (plt_data >= -20)]
    
    # 超过可视化区域 >20 橙色
    plt_data2 = plt_data.loc[plt_data > 20]
    # 超过可视化区域 <20 紫色
    plt_data3 = plt_data.loc[plt_data < -20]

    plt.cla()

    plt.xlabel('区域ID', fontsize=20)
    plt.ylabel('下车-上车', fontsize=20)
    plt.xlim([0, 264])
    plt.ylim([-20, 20])

    plt.title(res_times[i], fontsize=30)

    bar0 = plt.bar(x=plt_data0.index.values, height=plt_data0, color='r')
    bar1 = plt.bar(x=plt_data1.index.values, height=plt_data1, color='b')
    bar2 = plt.bar(x=plt_data2.index.values, height=plt_data2, color='orange')
    bar3 = plt.bar(x=plt_data3.index.values, height=plt_data3, color='purple')
    # print(plt_data3.index.values)
    for a, b in zip(plt_data2.index.values, plt_data2):
        text0 = plt.text(a - 6, 18.5, '%.0f'%b, ha = 'center', va = 'bottom', fontsize=20, color='orange')
    for a, b in zip(plt_data3.index.values, plt_data3):
        text1 = plt.text(a - 6, -20, '%.0f'%b, ha = 'center', va = 'bottom', fontsize=20, color='purple')

    plt.pause(pause)

plt.ioff()

plt.show()

由于matplotlib可视化的展示的局限性及不够美观，我们使用pyecharts作为可视化工具

`pyecharts热力图`

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import HeatMap
from pyecharts.charts import Timeline

In [ ]:
filename = "data/diff/2019-01.csv"
data = pd.read_csv(filename)

days = int(data.shape[0] / 24)
data['date'] = pd.to_datetime(data['time']).dt.date

tl = (
    Timeline(init_opts=opts.InitOpts(width='1500px',height='800px'))
    .add_schema(play_interval=1000,is_auto_play=False)
    )
    
x = [i for i in range(265)]
y = [i for i in range(24)]
for i in range(days):
    value = [[k, j, int(data.iloc[i*24+j][k+1])] for k in range(265) for j in range(24)]
    c = (
        HeatMap()
        .add_xaxis(x)
        .add_yaxis(str(data.iloc[i*24]['date']), y, value)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2019年1月全部地区热力图"),
            visualmap_opts=opts.VisualMapOpts(min_=-30, max_=30),
        )
    )
    tl.add(c, "{}日".format(i+1))
tl.render("graph/heatmap_all.html")

由于全部的数据过于庞大，我们配置的限制和观测的不够清楚我们希望找到局部帮助我们观察，我们看到红蓝色居于比较显著的有两个部分，130-160，220-265部分，不过我们先不急着确定

### 区域间旅程距离

计算一个265x265的数组表示各个区域间平均旅程距离

In [ ]:
filename = "data/clear_data/new_yellow_tripdata_2020-06.csv"
data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])[['trip_distance', "PULocationID", "DOLocationID"]]
data.head()

统计一个月记录下的平均距离

In [ ]:
def cal_dist(filename, write = True):
    year_month = filename.split("_")[-1].split(".")[0]

    data = pd.read_csv(filename).drop(columns=['Unnamed: 0'])[['trip_distance', "PULocationID", "DOLocationID"]]

    dist = np.zeros((265, 265))
    cnt = np.zeros((265, 265), dtype=np.int)

    print("Calculating file %s ..." % filename)

    start = time.time()
    for i in range(data.shape[0]):

        if (i + 1) % 200000 == 0:
            print("%d: cost time %.2fs" %(i + 1, time.time() - start))
            start = time.time()

        tmp = data.loc[i]
        d = tmp["trip_distance"]
        id1 = int(tmp["PULocationID"]) - 1
        id2 = int(tmp["DOLocationID"]) - 1
        
        dist[id1, id2] = dist[id1, id2] + d
        dist[id2, id1] = dist[id1, id2]
        cnt[id1, id2] = cnt[id1, id2] + 1
        cnt[id2, id1] = cnt[id1, id2]

    cnt[np.where(cnt == 0)] = 1
    res = dist / cnt

    res = pd.DataFrame(res)
    if write:
        res.to_csv("data/dist/%s.csv" % (year_month), index=False)
    print("Done!")
    
    return res

计算一个月740w条数据约20分钟

In [ ]:
i = 0
for root, dirs, files in os.walk("data/clear_data"):
    for f in files:
        if i == 2: # 只读取几个文件
            break
        i += 1
        filename = os.path.join(root, f)
        start = time.time()
        cal_dist(filename)
        print("Cost time: %.2fs" % (time.time() - start))
        print("--------" * 10)

由于数据量过于庞大，我们先只关注两个月的数据，将2019-01与2019-02的数据做差，用pyecharts画出热力图，观察哪一部分的数据变化的最小

In [ ]:
dist2 = pd.read_csv("data/dist/2019-02.csv")
dist1 = pd.read_csv("data/dist/2019-01.csv")
dist = dist1-dist2

In [ ]:
# 绘制地区之间距离热度图
from pyecharts import options as opts
from pyecharts.charts import HeatMap

x = [i for i in range(265)]
y = [i for i in range(265)]
value = [[i, j, int(dist.iloc[i][j])] for i in range(265) for j in range(265)]
c = (
    HeatMap(init_opts=opts.InitOpts(width='1000px',height='900px'))
    .add_xaxis(x)
    .add_yaxis("distance", y, value)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各地区距离"),
        visualmap_opts=opts.VisualMapOpts(min_=-30, max_=30),
    )
    .render("graph/heatmap_diff.html")
)

我们发现220-265部分变化会更小，所以我们最终选择这部分数据做分析及模型预测

部分地区热度图观察

In [ ]:
# 绘制一个月内部分区域的出租车热度图
filename = "data/diff/2019-01.csv"
data = pd.read_csv(filename)

days = int(data.shape[0] / 24)
data['date'] = pd.to_datetime(data['time']).dt.date

tl = (
    Timeline(init_opts=opts.InitOpts(width='1500px',height='800px'))
    .add_schema(play_interval=200,is_auto_play=False)
    )
    
x = [i for i in range(220,265)]
y = [i for i in range(24)]
for i in range(days):
    value = [[k, j, int(data.iloc[i*24+j][k+221])] for k in range(45) for j in range(24)]
    c = (
        HeatMap()
        .add_xaxis(x)
        .add_yaxis(str(data.iloc[i*24]['date']), y, value)
        .set_global_opts(
            title_opts=opts.TitleOpts(title="2019年1月部分地区热力图"),
            visualmap_opts=opts.VisualMapOpts(min_=-30, max_=30),
        )
    )
    tl.add(c, "{}日".format(i+1))
tl.render("graph/heatmap.html")

对上述部分进行分析：

* 某些地区早上红色、晚上蓝色，可能是办公楼
* 某些地区早上蓝色、晚上红色，可能是住宅区
* ...

## 模型搭建

通过观察分析，对我们挑选出的一些数据（由于全数据集较大）进行模型搭建：

区域ID：220-264

`模型`

一个图表示

* 节点：上下车的差值
  * 正，表示下车的出租数量更多（假设为空车数）
  * 负，表示上车的出租数量更多（假设为需求数）
* 边：出租车建议去下个区域的推荐值
  * 主要由我们统计得到的区域间距离及该时间段该区域的上下车差值估计的推荐值

### 数据获取

选定区域间trip距离的估计我们认为随时间变化不大，所以仅选择了2019-01月作为代表

In [ ]:
data = pd.read_csv("data/dist/2019-01.csv").iloc[220:265, 220:265]
data.to_csv("data/model/dist.csv", index = False)

同理，2019-01，月平均每一个小时上下车差值

In [ ]:
data = pd.read_csv("data/diff/2019-01.csv").iloc[:, 221:]
days = data.shape[0] // 24
mean = data.iloc[0:24].values
for i in range(1,days):
    tmp = data.iloc[i*24: (i + 1) * 24].values
    mean = mean + tmp
mean = mean / days
mean = pd.DataFrame(mean)
mean.columns = range(220,265)
mean.to_csv("data/model/diff.csv", index = False)

### 边的权重

推荐值

$
weight_{i, j} = 
\begin{cases}
\left|{\frac{diff_i \times diff_j}{dist_{i,j}}}\right|, & if\quad diff_i > 0\quad and\quad diff_j < 0 \\
0, & otherwise
\end{cases}
$

$weight_{i, j}$表示有向边i -> j的权重，$diff_i$表示节点i的值（空车数），$diff_j$表示节点j的值（需求数），$dist_{i, j}$表示节点i到j的旅途距离

In [ ]:
dist = pd.read_csv("data/model/dist.csv")
diff = pd.read_csv("data/model/diff.csv")

In [ ]:
res = []
for t in range(24):
    weights = np.zeros((45, 45))
    for i in range(45):
        for j in range(45):
            diff_i = diff.iloc[t, i]
            diff_j = diff.iloc[t, j]
            dist_i_j = dist.iloc[i, j]
            if diff_i > 0 and diff_j < 0 and dist_i_j != 0:
                weights[i, j] = abs(diff_i * diff_j / dist_i_j)
                # print(diff_i, diff_j, dist_i_j, weights[i, j])
            else:
                continue
    res.append(weights)

res = np.array(res)
np.save('data/model/graph.npy', res)

我们得到了一个矩阵存储需要的边权重

节点的值由dist得到

`pyecharts关系图`

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Timeline
from pyecharts.charts import Graph

判断节点颜色函数

In [ ]:
def node_color(n):
    if n < 0:
        return "blue"
    if n > 0:
        return "red"
    return "yellow"

In [ ]:
from pyecharts import options as opts
from pyecharts.charts import Graph

tl = (Timeline(init_opts=opts.InitOpts(width="1600px", height="800px"))).add_schema(play_interval=500,is_auto_play=False)

node = pd.read_csv("data/model/diff.csv")
edge = np.load("data/model/graph.npy")

for k in range(24):
    nodes = [
        {
            "name": str(i+220),
            "value": node.iloc[k,i],
            "symbolSize": math.log(1+abs(node.iloc[k,i]),1.08),
            "itemStyle": {"normal": {"color": node_color(node.iloc[k,i])}},
        }
        for i in range(45)
    ]
    edges = []
    for i in range(45):
        for j in range(45):
            if edge[k,i,j] != 0:
                edges.append({"source":str(i+220), "target":str(j+220), "value":edge[k,i,j]})
    c = (
        Graph(init_opts=opts.InitOpts(width="1600px", height="800px"))
        .add(
            series_name="",
            nodes=nodes,
            links=edges,
            layout="circular",
            is_roam=True,
            is_focusnode=True,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=0.5, curve=0.3, opacity=0.7),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="预测关系图"))
    )
    tl.add(c, "{}时".format(k))
tl.render("graph/predict.html")

## 附录

使用下列命令查看matplotlib使用的后端

In [ ]:
%pylab